In [1]:
import sys
sys.path.append('../../FourVector')
sys.path.append('../project')

from FourVector import FourVector
from ThreeVector import ThreeVector

from FutureColliderTools import SmearVertex, GetCorrectedMass, GetMissingMass2, GetQ2
from FutureColliderDataLoader import LoadData_KMuNu, LoadData_DsMuNu
from FutureColliderVariables import *
from FutureColliderFitter import FutureFitter

import numpy as np
import ROOT

ROOT.enableJSVis()
ROOT.gStyle.SetOptStat(0)

Welcome to JupyROOT 6.08/02


In [57]:
File_Fits = ROOT.TFile.Open("../output/FitterOutput.root", "READ")

Canv_Fits = {}
Canv_Fits["Ds"] = ROOT.TCanvas("c_Ds", "", 900, 900)
Canv_Fits["K"]  = ROOT.TCanvas("c_K", "", 900, 900)
Canv_Fits["Ds"].Divide(3,3)
Canv_Fits["K"] .Divide(3,3)

Legends = {}
Texts = []

for Mode in ["K", "Ds"]:
    it = 1
    for resolution in np.linspace(0.3, 1.0, 8):
        canvas = File_Fits.Get("c_Fit_{mode}_{resolution}".format(mode = Mode, resolution = resolution))
        Canv_Fits[Mode].cd(it)
        for prim in canvas.GetListOfPrimitives():
            if prim.GetName() == "TPave":
                Legends[Mode] = prim.Clone()
                canvas.GetListOfPrimitives().Remove(prim)
                break
        Text = ROOT.TPaveText(0.02, 0.8, 0.3, 0.98, "NDC")
        Text.AddText("{0} #sigma LHCb".format(resolution))
        Text.SetBorderSize(0)
        Text.SetFillColor(0)
        canvas.DrawClonePad()
        
        Text.Draw()
        Texts += [Text]
        it+=1
        
    Canv_Fits[Mode].cd(9)
    Legends[Mode].SetX1(0)
    Legends[Mode].SetX2(1)
    Legends[Mode].SetY1(0)
    Legends[Mode].SetY2(1)
    Legends[Mode].Draw()


In [58]:
Canv_Fits["Ds"].Draw()

In [59]:
Canv_Fits["K"].Draw()

In [40]:
import pickle
Results = pickle.load( open("../output/FitResults.p") )

print "Results for K Mu Nu Fit:"
print Results["K"]
print "\nResults for Ds Mu Nu Fit:"
print Results["Ds"]

Results for K Mu Nu Fit:
+---------------------+----------------+---------------+
| Vertex Error / LHCb |  Signal Yield  |   Error [%]   |
+---------------------+----------------+---------------+
|         0.2         | 0.173737269939 |  2.1949514792 |
|         0.3         | 0.16000528803  | 2.12937142758 |
|         0.4         | 0.142673527895 | 2.68048705797 |
|         0.5         | 0.137956552735 | 2.96918362306 |
|         0.6         | 0.126890984762 | 3.15037778533 |
|         0.7         | 0.118921706116 | 3.44075033928 |
|         0.8         | 0.115883494019 | 3.80907254048 |
|         0.9         | 0.105570368992 | 4.89601870493 |
|         1.0         | 0.102834094838 | 4.84876883524 |
+---------------------+----------------+---------------+

Results for Ds Mu Nu Fit:
+---------------------+----------------+---------------+
| Vertex Error / LHCb |  Signal Yield  |   Error [%]   |
+---------------------+----------------+---------------+
|         0.2         | 0.2634965726

In [12]:
Graph_K  = File_Fits.Get("Error_Budget_K")
Graph_Ds = File_Fits.Get("Error_Budget_Ds")

Graph_K.SetLineStyle(2)
Graph_K.SetMarkerStyle(4)

Graph_Ds.SetLineStyle(2)
Graph_Ds.SetMarkerStyle(4)


canv_Budget = ROOT.TCanvas("c_graph", "", 900, 900)
Graph_K.Draw()
canv_Budget.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_graph


In [13]:
canv_Budget.Clear()
Graph_Ds.Draw()
canv_Budget.Draw()